In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import training as training
import connect4cnn as connect4cnn
importlib.reload(training);
importlib.reload(connect4cnn)
from connect4cnn import Connect4Cnn


In [2]:
#
# Hyper parameter
#
gamma = 0.9 # Q-learning's discount factor, should probably stay constant at 0.9
learning_rate = 0.01 # initial learning rate
actual_learning_rate = 0.01 # used to reset optimizers from file
epsilon = 0.01
validation_games = 5000
omega = 1 # the percentage of opponent random moves during validation


In [3]:

#
# Create the model and optimizer
# 
games = 0
model = Connect4Cnn()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
sum(p.numel() for p in model.parameters() if p.requires_grad)

244487

In [4]:
#
# Load model from checkpoint
#
games = 1000000
training.loadCheckpoint(model, optimizer, f'connect4-{games}', actual_learning_rate)

In [ ]:
#
# TRAINING
#
gamesToGo = 500000
training.train(model, optimizer, gamesToGo, epsilon, omega, gameOffset = games, gamma = gamma)
games += gamesToGo

### Connect4 CNN training 
#### Layers: C64-C128-P2/1-C128-P2/1
optimizer: SGD initial lr: 0.01  
validation 10000 games against normalized softmax moves and &omega; random moves  
exploration via normalized softmax and &epsilon;  
loss over last 50.000 games

| Games     | lr        | &epsilon; | Loss                  | &omega;   |Cross      | Circle    | Remarks
| :-------: | :-:       | :---:     | :-----------:         | :----:    |:----:     | :-----:   | :-------:
|           | 0.01      | 0.01      |                       | 1         | 87.94%    | 78.86%    |
| 50.000    |           |           | 0.015758221467523835  |           | 96.16%    | 93.56%    |
| 100.000   |           |           | 0.011589428953352762  |           | 99.46%    | 98.06%    |
| 150.000   |           |           | 0.009629916073209097  |           | 99.62%    | 98.06%    | 
| 200.000   |           |           | 0.008917801845750037  |           | 99.74%    | 98.92%    |
| 250.000   |           |           | 0.008638794834900126  |           | 99.66%    | 98.86%    |
| 300.000   |           |           | 0.008321861519028006  |           | 99.74%    | 99.10%    |
| 350.000   |           |           | 0.008074597970579352  |           | 99.84%    | 99.04%    |
| 400.000   |           |           | 0.007874082113285257  |           | 99.78%    | 99.58%    |
| 450.000   |           |           | 0.007648168640614303  |           | 99.82%    | 99.26%    |
| 500.000   |           |           | 0.00745036941245753   |           | 99.84%    | 99.60%    |
| 550.000   |           |           | 0.007112492493568134  |           | 99.88%    | 99.50%    |
| 600.000   |           |           | 0.00682966207332689   |           | 99.96%    | 99.60%    |
| 650.000   |           |           | 0.006467445320635861  |           | 99.96%    | 99.56%    |
| 700.000   |           |           | 0.006305591097399774  |           | 99.98%    | 99.74%    |
| 750.000   |           |           | 0.006043971638668627  |           | 99.96%    | 99.80%    |
| 800.000   |           |           | 0.00682966207332689   |           | 99.96%    | 99.60%    |
| 850.000   |           |           | 0.005662222434448868  |           | 99.98%    | 99.86%    |
| 900.000   |           |           | 0.005508421725580128  |           | 99.92%    | 99.88%    |
| 950.000   |           |           | 0.005368859568469625  |           | 99.96%    | 99.86%    |
| 1.000.000 |           |           | 0.005204164923570288  |           | 99.96%    | 99.80%    |


In [ ]:
training.validate(model, validation_games, omega)